In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def add_layer(inputs, in_size, out_size,nth_layer, activation_function = None):
    # add one more layer adn return the output of this layer
    layer_name = "layer%s" % nth_layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_uniform([in_size, out_size]),name='W')
            tf.summary.histogram(layer_name + '/weights', Weights)
            
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size])+ 0.1, name='b')  # 不推荐设置为0
            tf.summary.histogram(layer_name + '/biases', biases)
            
        with tf.name_scope("Wx_plus_b"):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
            
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
          
        tf.summary.histogram(layer_name + '/outputs', outputs)
        return outputs

In [3]:
x_data = np.linspace(-1,1,300)[:,np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise
#print(x_data)

In [4]:
# define placeholder for inputs to network
with tf.name_scope("inputs"):
    xs = tf.placeholder(tf.float32,[None,1], name='x_input')
    ys = tf.placeholder(tf.float32, [None,1], name='y_input')

In [5]:
# 假设隐层有10个神经元，输出层有1个神经元
layer1 = add_layer(xs, 1, 10, nth_layer=1, activation_function=tf.nn.relu)
# 输出层
predition = add_layer(layer1, 10, 1, nth_layer=2, activation_function=None)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - predition),
                                        reduction_indices=[1]))
    tf.summary.scalar("loss", loss)
    
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

In [10]:
sess = tf.Session()
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("logs/",sess.graph)

In [11]:
init = tf.global_variables_initializer()
sess.run(init)

In [12]:
for i in range(2000):
    sess.run(train_step, feed_dict={xs:x_data,ys:y_data})
    if i % 50 == 0:
        result = sess.run(merged,feed_dict={xs:x_data,ys:y_data})
        writer.add_summary(result,i) # result 和步数